In [ ]:
TODO: 

In [56]:
import pandas as pd 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
#%matplotlib notebook

from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


In [57]:
# datasets provided by https://github.com/llSourcell/recommender_live
dataset1 = '/home/kristina/Downloads/10000.txt'
dataset2 = '/home/kristina/Downloads/song_data.csv'

temp_df1 = pd.read_csv(dataset1, sep="\t", names =["user_id", "song_id", "listen_count"])
temp_df2 = pd.read_csv(dataset2)

final_df = pd.merge(temp_df1, temp_df2)
final_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,930d2be6c85315d72cab9823ec0f7bfe7e477794,SOBBMDR12A8C13253B,1.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007


In [58]:
train_data, test_data = cv.train_test_split(final_df, test_size=0.25)

In [60]:
subset_df = train_data[['user_id','song_id', 'listen_count']]
matrix_df = pd.crosstab(subset_df.user_id, subset_df.song_id, values = subset_df.listen_count, aggfunc="first")
matrix_df.fillna(0, inplace=True)

magnitude = np.sqrt(np.square(matrix_df).sum(axis=1))
matrix_df = matrix_df.divide(magnitude, axis='index')

data_sparse = sparse.csr_matrix(matrix_df)
similarities = cosine_similarity(data_sparse.transpose())
sim = pd.DataFrame(data=similarities, index=matrix_df.columns, columns= matrix_df.columns)

In [61]:
#item-item collaboration
data = sim.loc['SOBXHDL12A81C204C0'].nlargest(50)
df1 = final_df.loc[final_df['song_id'].isin(data.index.values)]
df1 = df1.drop_duplicates(subset=['song_id'], keep="first")

df1.head(n = 10)

,user_id,song_id,listen_count,title,release,artist_name,year
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007
945,b64cdd1a0bd907e5e00b39e345194768e330d652,SOALEQA12A58A77839,1.0,Jumping Jack Flash,Shine A Light,The Rolling Stones,1982
1454,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOCAHRT12A8C13A1A4,5.0,S.O.S.,SOS,Jonas Brothers,2007
2598,5a905f000fc1ff3df7ca807d57edb608863db05d,SOYEWFR12AB0189AB6,1.0,Odessa,Odessa,Caribou,2010
2816,baf47ed8da24d607e50d8684cde78b923538640f,SOKHZWA12A8C13672E,7.0,Older Gods,Wu: The Story Of The Wu-Tang Clan,Wu-Tang Clan,1997
3133,bd8475385f0aa78830fa6dfce9e7242164b035c8,SOEMUXL12A58A7B848,1.0,Rehab,Hip Hop The Collection 2009,Rihanna,2007
3270,bd8475385f0aa78830fa6dfce9e7242164b035c8,SOSDTGC12AF72A2BC3,1.0,Chinese,It's Not Me_ It's You,Lily Allen,2009
4174,a58de017cbeda1763ea002fe027ed41b4ed53109,SOUZLXN12A6D4F4A66,1.0,Back To Black,Back To Black,Amy Winehouse,2006
4977,99b5f915dd56a0150eb91605e1f6109e1e80c9de,SOKAJXI12AB017DEC7,1.0,Brimstone Landscapes,Fragmentary Evidence,Augury,2009
5337,732f88be38fae217f8ab7e24c20dd072436e3e40,SOMUWFQ12AB0184608,1.0,Lookin' Out My Back Door,The Singles Collection,Creedence Clearwater Revival,1970
